
# Natural Language Generation using Markovify

This project makes use of the Markovify package within python to develop a language generation model. Markovify operates on the ideas of Markov chains which trains a language model based on local context.

(Please make sure to change your runtime type to GPU before running to avoid long wait times)

We are mounting our google drive below so we can bring in our dataset.
Following that we are going to install all the necessary packages (Markovify, openai, transformers, and language_tool_pyton)




In [ ]:
#This part of the code does not need to be ran if you are not reading the excel file from your google drive
from google.colab import drive
drive.mount('/gdrive')
!ln -s "/gdrive/MyDrive/AI Introduction" "/content/AI Introduction/Datasets"

Mounted at /gdrive
ln: failed to create symbolic link '/content/AI Introduction/Datasets': No such file or directory


In [ ]:
#Installing our dependencies
!pip install markovify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 22.3 MB/s eta 0:00:00
  Created wheel for markovify: filename=markovify-0.9.4-py3-none-any.whl size=18625 sha256=6b9d0c4a4ab2139b94c120db1129157c20c45e24707af99cfcb316e01f509ec9
  Stored in directory: /root/.cache/pip/wheels/76/0a/ab/8727d219981e57e6036316dd2ec2037e61ccea0c016f7ae0c1
Successfully built markovify


In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 28.1 MB/s eta 0:00:00


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 22.5 MB/s eta 0:00:00


In [ ]:
!pip install language_tool_python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# pip install tensorflow-estimator==2.12.0

# Import all our packages needed

In [ ]:
import markovify
import pandas as pd
import re
import tensorflow_hub as hub
import tensorflow as tf
import language_tool_python
import spacy
import nltk
from sklearn.model_selection import train_test_split
from transformers import pipeline
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Data preprocessing and Model training

We are removing the @ handles for users so we don't end up tweeting actual people by mistake. We are also creating our model and embedding a pre-trained sentence model here for coherence calculations.

twcs.csv - (Around 3 minutes to train the model using GPU on the training dataset which contains 2.8 million records)

preprocessed_1mill.csv - (Around 1-2 minutes to train the model using GPU on the training dataset which contains 1.6 million records)

Due to runtime issues, we will be reducing our dataset to a size of 400,000 records instead of 1.6 million. This could be fixed in the future if there is access to Colab Plus or if there is more ram for use.

In [ ]:
# Load the dataset about tweets
cs_ds = pd.read_csv('/gdrive/MyDrive/AI Introduction/Datasets/preprocessed_tweets_1mill.csv')

cs_ds['tweet'] = cs_ds['tweet'].apply(lambda x: re.sub(r'@\w+', '', x))

#Splicing our dataset so that it isn't too big
cs_ds = cs_ds.sample(n=400000, random_state=42)

# Split the dataset into training and test sets
train_ds, test_ds = train_test_split(cs_ds, test_size=0.2, random_state=42)

# Tokenize our tweet column and split up each word
train_ds['sentences'] = train_ds['tweet'].apply(sent_tokenize)
test_ds['sentences'] = test_ds['tweet'].apply(sent_tokenize)

train_sentences = [sentence for sentences in train_ds['sentences'] for sentence in sentences]
train_text = '\n'.join(train_sentences)

model_1 = markovify.NewlineText(train_text, state_size=1)
model_2 = markovify.NewlineText(train_text, state_size=2)
model_3 = markovify.NewlineText(train_text, state_size=3)

# # Train three Markov models on the training set with different state sizes
# model_1 = markovify.NewlineText(train_ds.tweet, state_size=1)
# model_2 = markovify.NewlineText(train_ds.tweet, state_size=2)
# model_3 = markovify.NewlineText(train_ds.tweet, state_size=3)

# Load a pre-trained sentence embedding model
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

train_ds['tweet'].head

<bound method NDFrame.head of 1441529    the full moon tonight is amazing almost like d...
667231                         buttercupamy i miss amy cards
869896     at work and reading peoples 100 most beautiful...
1194774    likelyto get how are you doing thanks for foll...
1082098                                   kinky now i get it
                                 ...                        
1135941    at a rooftop party ln brooklyn hey at least ln...
31275      time for the school and work week to start  it...
1522785    aureliustjin  haha yeah your name is very uniq...
387960                              i need money for tickets
472226                            stevem4y i need dj lessons
Name: tweet, Length: 320000, dtype: object>

This section is to just get an idea of our datasets.

In [ ]:
len(cs_ds)
len(train_ds)
len(test_ds)
print("Full dataset:", len(cs_ds), "\nTraining", len(train_ds), "\nTesting", len(test_ds))

# User Input
Here we want to prompt the user for an input so that we can start generating the sentences they want. The idea behind this is we want our generated sentences to have a subject in mind.

We are generating a set of ten sentences for each model since each model is trained differently. The reason for this is because we want to compare the results of each of the models to find the most optimal model that could pass off as a "human".

We are also creating our sentiment classification pipeline here through the use of the transformers package (This will tell us the tone of our generated text).
Our grammar metric evaluation method is also imported in during this step as it comes from a pre-trained dictionary.

twcs_csv - Generating sentences takes around 6 mins to run.

preprocessed_1mill - Generating sentences takes around 2 mins to run.

In [ ]:
# Get user input for the seed text
seed_text = input("Enter a seed text: ")

# Generate 10 sentences using each of the three Markov models
generated_sentences = []
for i in range(10):
    # Use model 1 to generate a sentence
    sentence_1 = None
    for j in range(100):
        sentence_1 = model_1.make_sentence(tries=10000, max_overlap_ratio=0.7)
        if sentence_1 is not None and seed_text.lower() in sentence_1.lower():
            break
    if sentence_1 is not None:
        generated_sentences.append("Model 1: " + sentence_1)

    # Use model 2 to generate a sentence
    sentence_2 = None
    for j in range(100):
        sentence_2 = model_2.make_sentence(tries=10000, max_overlap_ratio=0.6)
        if sentence_2 is not None and seed_text.lower() in sentence_2.lower():
            break
    if sentence_2 is not None:
        generated_sentences.append("Model 2: " + sentence_2)

    # Use model 3 to generate a sentence
    sentence_3 = None
    for j in range(100):
        sentence_3 = model_3.make_sentence(tries=10000, max_overlap_ratio=0.5)
        if sentence_3 is not None and seed_text.lower() in sentence_3.lower():
            break
    if sentence_3 is not None:
        generated_sentences.append("Model 3: " + sentence_3)

# Initialize a sentiment analysis pipeline
sentiment_classifier = pipeline("sentiment-analysis")

# Initialize the LanguageTool API
grammar_checker = language_tool_python.LanguageTool('en-US')

Enter a seed text: movie


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


INFO:language_tool_python.download_lt:Unzipping /tmp/tmpdsmd1w3y.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-5.7.zip to /root/.cache/language_tool_python.


# Displaying sentences

We want to display the sentences to the user so that they can see if they want to keep these sentences or not.

In [ ]:
generated_sentences

['Model 1: lexiewohlfort i eat a movie so hot weather',
 'Model 2: erickweber the movie will be in the sun better get off soon',
 'Model 3: arrrrrggghhh life is so different and better now theres three of us are kiddos and im glad i aint got that special once to spend it doing work i shouldve already done',
 'Model 1: whats up tomorrow via illness',
 'Model 2: grrrr ran out of twitterlandthe day is today time to get back in this crazy magic that was an awesome christian ignorance mostly but still good 4 u',
 'Model 3: heading to yoga then gb family picnic tomorrow ah what a beautiful movie like wow now calling it a day freakin emotions are dead tired',
 'Model 1: pharout85 just bought in the place my birthday party so inexpensive i see them dunno wed dont see',
 'Model 2: plasticmartyr oreos and skittles so i dunno her name',
 'Model 3: its mid of june clowdy weather in pune but i will never see this classic character on television ever again',
 'Model 1: omg going to the movie awards 

In [ ]:
#Only reason for this is because we kept getting "deprecated" messages about the .append feature for Pandas, but when we used the .concat method
#we would encounter errors running the code.
import warnings
warnings.filterwarnings('ignore')

# Evaluation Metrics

In the section below we will be calculating all of our metric evaluation scores.



*   Coherence: This will measure the flow and consistency of the generated text.
*   Sentiment: Explains the nature of the sentence and the general emotion behind it (ex. Positive or Negative)
*   Sentiment Confidence: This score represents how confident the evaluations assumes the predicited nature to be.
*   Grammar: General flow and grammar of the sentence.
*   Diversity: Measures the variety of our generated tweets in terms of vocabulary, phrasing, and context.

On top of this we are calculating the average scores for each of the numerical evaluation metrics as this will give us a better idea of the general performance for each of the models.

After all is done, we will be spitting out all the information into a dataframe for easy storage and easy viewing.







In [ ]:
coherence_scores = {'model_1': [], 'model_2': [], 'model_3': []}
grammaticality_scores = []
diversity_scores_uni = []
diversity_scores_bi = []
diversity_scores_tri = []

results_df = pd.DataFrame(columns=["Model Type", "Sentence", "Sentiment", "Sentiment Conf.","Coherence Score", "Grammaticality Score", "Diversity Score"])

for sentence in generated_sentences:
    if sentence is not None:
        # Compute the coherence score for the generated sentence using the test set
        input_text = [sentence]  # convert the input to a list of strings
        input_tensor = tf.constant(input_text, dtype=tf.string)  # convert to a tensor
        embedding = embed(input_tensor).numpy()  # compute the embedding

        # Compute the cosine similarity between the embedding of the generated sentence and the embeddings of the test set
        test_embeddings = embed(test_ds.tweet.tolist()).numpy()
        similarity = embedding.dot(test_embeddings.T)

        coherence_score = similarity.max()

        # Compute the grammaticality score for the generated sentence
        matches = grammar_checker.check(sentence)
        grammaticality_score = len(matches)
        grammaticality_scores.append(grammaticality_score)

        ngrams = set()
        for j in range(len(sentence.split())):
            uni = sentence.split()[j]
            ngrams.add(uni)
        diversity_score_uni = len(ngrams) / len(sentence.split())
        diversity_scores_uni.append(diversity_score_uni)

        ngrams = set()
        for j in range(len(sentence.split()) - 1):
            bigram = " ".join(sentence.split()[j:j+2])
            ngrams.add(bigram)
        diversity_score_bi = len(ngrams) / (len(sentence.split()) - 1)
        diversity_scores_bi.append(diversity_score_bi)
        
        ngrams = set()
        for j in range(len(sentence.split()) - 2):
            trigram = " ".join(sentence.split()[j:j+3])
            ngrams.add(trigram)
        diversity_score_tri = len(ngrams) / (len(sentence.split()) - 2)
        diversity_scores_tri.append(diversity_score_tri)

        # Append the coherence score to the appropriate model's list
        diversity_score = None
        n_grams = None
        model_type = None
        if "Model 1" in sentence:
            coherence_scores['model_1'].append(coherence_score)
            diversity_score = diversity_score_uni
            n_grams = "1"
            model_type = "Model 1"
        elif "Model 2" in sentence:
            coherence_scores['model_2'].append(coherence_score)
            diversity_score = diversity_score_bi
            n_grams = "2"
            model_type = "Model 2"
        elif "Model 3" in sentence:
            coherence_scores['model_3'].append(coherence_score)
            diversity_score = diversity_score_tri
            n_grams = "3"
            model_type = "Model 3"

        # Print the results for this sentence
        result = sentiment_classifier(sentence)[0]
        sentiment = result['label']
        sentiment_score = result['score']
        print(f"Sentence: {sentence}\nSentiment: {sentiment}\nSentiment Conf.: {sentiment_score}\nCoherence score: {coherence_score}\nGrammaticality score = {grammaticality_score}\nDiversity score when n = {n_grams}: {diversity_score}\n")


        # Append the data to the DataFrame
        results_df = results_df.append({
            "Model Type": model_type,
            "Sentence": sentence,
            "Sentiment": sentiment,
            "Sentiment Conf.": sentiment_score,
            "Coherence Score": coherence_score,
            "Grammaticality Score": grammaticality_score,
            "Diversity Score": diversity_score
        }, ignore_index=True)

# Compute the average coherence score for each model and print the results
for model, scores in coherence_scores.items():
    if len(scores) > 0:
        avg_score = sum(scores) / len(scores)
        print(f"Average coherence score for {model}: {avg_score}")
    else:
        print(f"No generated sentences for {model}.")

average_grammaticality_score = sum(grammaticality_scores) / len(grammaticality_scores)
print("\nAverage grammaticality score:", average_grammaticality_score)

# Compute the average diversity scores for each model
model_1_diversity = (sum(diversity_scores_uni) / len(diversity_scores_uni))
model_2_diversity = (sum(diversity_scores_bi) / len(diversity_scores_bi))
model_3_diversity = (sum(diversity_scores_tri) / len(diversity_scores_tri))

print(f"\nAverage Diversity Score (uni_gram) for Model 1: {model_1_diversity}")
print(f"Average Diversity Score (bi_gram) for Model 2: {model_2_diversity}")
print(f"Average Diversity Score (tri_gram) for Model 3: {model_3_diversity}")

Sentence: Model 1: lexiewohlfort i eat a movie so hot weather
Sentiment: NEGATIVE
Sentiment Conf.: 0.9907009601593018
Coherence score: 0.5039879083633423
Grammaticality score = 2
Diversity score when n = 1: 1.0

Sentence: Model 2: erickweber the movie will be in the sun better get off soon
Sentiment: NEGATIVE
Sentiment Conf.: 0.9989097118377686
Coherence score: 0.48291417956352234
Grammaticality score = 1
Diversity score when n = 2: 1.0

Sentence: Model 3: arrrrrggghhh life is so different and better now theres three of us are kiddos and im glad i aint got that special once to spend it doing work i shouldve already done
Sentiment: POSITIVE
Sentiment Conf.: 0.8538222312927246
Coherence score: 0.5284404754638672
Grammaticality score = 5
Diversity score when n = 3: 1.0

Sentence: Model 1: whats up tomorrow via illness
Sentiment: NEGATIVE
Sentiment Conf.: 0.9849321246147156
Coherence score: 0.41483747959136963
Grammaticality score = 1
Diversity score when n = 1: 1.0

Sentence: Model 2: grr

# Dataframe

In [ ]:
print(results_df)

NameError: ignored

Download the dataframe to csv format. Just uncomment the code and you are good to go.

In [ ]:
# from google.colab import files
# results_df.to_csv('output.csv', encoding = 'utf-8-sig') 
# files.download('output.csv')

# Twitter API 

This will be used to access the twitter API so you could post one of the generated messages if you wanted to.

In [ ]:
from requests_oauthlib import OAuth1Session
import os
import json

def postMessage(message):
    # In your terminal please set your environment variables by running the following lines of code.
    # export 'CONSUMER_KEY'='<your_consumer_key>'
    # export 'CONSUMER_SECRET'='<your_consumer_secret>'
    
    consumer_key = 'Hmzz4Hiqn8LA6FMpyVmwo5kQ8'
    consumer_secret = 'oJ4yBXiUK2eLGryqc1Xz0fprW1UZ0sscWEv7cCi9X4j0CEqAA7'
    
    # Be sure to add replace the text of the with the text you wish to Tweet. You can also add parameters to post polls, quote Tweets, Tweet with reply settings, and Tweet to Super Followers in addition to other features.
    payload = {"text": message}
    
    # Get request token
    request_token_url = "https://api.twitter.com/oauth/request_token?oauth_callback=oob&x_auth_access_type=write"
    oauth = OAuth1Session(consumer_key, client_secret=consumer_secret)
    
    try:
        fetch_response = oauth.fetch_request_token(request_token_url)
    except ValueError:
        print(
            "There may have been an issue with the consumer_key or consumer_secret you entered."
        )
    
    resource_owner_key = fetch_response.get("oauth_token")
    resource_owner_secret = fetch_response.get("oauth_token_secret")
    print("Got OAuth token: %s" % resource_owner_key)
    
    # Get authorization
    base_authorization_url = "https://api.twitter.com/oauth/authorize"
    authorization_url = oauth.authorization_url(base_authorization_url)
    print("Please go here and authorize: %s" % authorization_url)
    verifier = input("Paste the PIN here: ")
    
    # Get the access token
    access_token_url = "https://api.twitter.com/oauth/access_token"
    oauth = OAuth1Session(
        consumer_key,
        client_secret=consumer_secret,
        resource_owner_key=resource_owner_key,
        resource_owner_secret=resource_owner_secret,
        verifier=verifier,
    )
    oauth_tokens = oauth.fetch_access_token(access_token_url)
    
    access_token = oauth_tokens["oauth_token"]
    access_token_secret = oauth_tokens["oauth_token_secret"]
    
    # Make the request
    oauth = OAuth1Session(
        consumer_key,
        client_secret=consumer_secret,
        resource_owner_key=access_token,
        resource_owner_secret=access_token_secret,
    )
    
    # Making the request
    response = oauth.post(
        "https://api.twitter.com/2/tweets",
        json=payload,
    )
    
    if response.status_code != 201:
        raise Exception(
            "Request returned an error: {} {}".format(response.status_code, response.text)
        )
    
    print("Response code: {}".format(response.status_code))
    
    # Saving the response as JSON
    json_response = response.json()
    print(json.dumps(json_response, indent=4, sort_keys=True))

postMessage("generic Twitter rant")